# Perceptron multicapa: Clasificación
En este notebook trabajaremos sobre predicción de Churn. Específicamente, decidir si una persona dejará un banco o no dadas sus características usando una red neuronal fully connected.

In [ ]:
# importar librerias necesarias
import numpy as np
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

## Parte 1. Manejo de datos
### 1. Importar el conjunto de datos / EDA

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
print(dataset.columns)
print(dataset.info())
print(dataset.head())

### 2. Procesamiento de datos
Primero removemos la variable dependiente `Exited` ya que esta representa la etiqueta. De esta manera separamos nuestros datos en atributos `X` y etiqueta `y`.

In [ ]:
# TODO: Selecciona las características y la variable objetivo
feature_cols = []
X = dataset[feature_cols]
y = dataset[]
print(X.shape, y.shape, y[:10])

### Ingeniería de atributos

Tenemos diferentes tipos de variables, algunas numéricas y otras de tipo categórico o `object` en pandas. Podemos usar diferentes tipos de codificadores para los categóricos:
- [OneHotEncoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder)
- [OrdinalEncoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder)

Otras transformaciones comunes son la normalización:
- [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


# Diccionario de encoders feature: encoder para poder procesar los datos
encoders = [
    # handle_unknown='ignore' para evitar errores si hay categorias nuevas en test set
    # unknown sera codificado como otra categoria
    ("gender_enc", OneHotEncoder(handle_unknown='ignore'), ["Gender"]),
    ("geo_enc", OrdinalEncoder(), ["Geography"]),
]

# Solo aplicamos los encoders a las columnas categóricas
# las demas columnas se dejan igual (remainder='passthrough')
preprocessor = ColumnTransformer(encoders, remainder='passthrough')
X_processed = preprocessor.fit_transform(X)

# Nota como X_processed es un numpy array, ya no tenemos los nombres de las columnas y no tenemos un pd dataframe
print(X_processed[:5])

In [ ]:
# Adicionalmente, normalizamos los datos numéricos
sc = StandardScaler()
X_processed = sc.fit_transform(X_processed)

print("Datos procesados y normalizados:")
print(X_processed[:5])

### 3. Uniendo todo el manejo de datos...

Para entrenar siempre tenemos que tener un split de train y validación. 
Vamos a primero generar el split y después aplicar el procesamiento de la celda anterior para evitar el data leakage.
En la siguiente celda se resume el codigo anterior

In [ ]:
from sklearn.model_selection import train_test_split

# leer datos
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset[feature_cols]
y = dataset['Exited']

# Separar en train y test, 20% de los datos van a validacion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Transformar a numpy arrays
y_train = y_train.values
y_test = y_test.values

# Preprocesamiento de datos
def preprocess(X, train=False, encoders=None, scaler=None):
    if train:
        # TODO: Elige y aplica los encoders
        encoders = [
            ("gender_enc", ... , ["Gender"]),
            ("geo_enc", ..., ["Geography"]),
        ]
        preprocessor = ColumnTransformer(encoders, remainder='passthrough')
        X_encoded = preprocessor.fit_transform(X)
        print(X_encoded[:1])
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_encoded)

        return X_processed, preprocessor, scaler
    else:
        X_encoded = encoders.transform(X)  # encoders = preprocessor ya entrenado
        X_processed = scaler.transform(X_encoded)
        return X_processed

X_train_processed, preprocessor, scaler = preprocess(X_train, train=True)

# Aplicar procesamiento de datos a validacion, ya no se usa fit_transform sino solo transform
# le mandamos los encoders y el normalizador entrenado
X_test_processed = preprocess(X_test, train=False, encoders=preprocessor, scaler=scaler)



Como visto en clase, los codificadores le asignan un valor numerico a un categórico. Para poder saber a que se asignó cada categórico corre la siguiente celda. También podras visualizar la información que guarda el normalizador.

#TODO: Responde a que columna(s) corresponde la geografía?

In [ ]:
def viz_encoders(preprocessor, scaler):
    print("Encoders usados:")
    for name, encoder, cols in preprocessor.transformers_:
        if name != 'remainder':
            print(f"\n- {name}: {encoder} en columnas {cols}")
            if hasattr(encoder, 'categories_'):  # OrdinalEncoder
                print(f"  Categorías: {encoder.categories_}")
            if hasattr(encoder, 'get_feature_names_out'):  # OneHotEncoder
                print(f"  Columnas codificadas: {encoder.get_feature_names_out(cols)}")
    
    print("\nScaler usado:")
    print(scaler)
    
    # Imprimir medias y desviaciones del StandardScaler
    if hasattr(scaler, 'mean_') and hasattr(scaler, 'scale_'):
        print("\nMedias del StandardScaler:")
        print(scaler.mean_)
        print("Desviaciones estándar del StandardScaler:")
        print(scaler.scale_)

viz_encoders(preprocessor, scaler)

def show_column_mapping(preprocessor):
    col_index = 0
    for name, transformer, cols in preprocessor.transformers_:
        if name != 'remainder':
            if hasattr(transformer, 'get_feature_names_out'):  # OneHot
                out_cols = transformer.get_feature_names_out(cols)
            else:  # Ordinal o 1 columna
                out_cols = cols
            for c in out_cols:
                print(f"Columna {col_index}: {c}")
                col_index += 1

    # Columns passthrough
    if preprocessor.remainder == 'passthrough':
        passthrough_cols = preprocessor.transformers_[-1][-1]  # lista de columnas pasadas
        # O si no se listan, agregarlas manualmente
        print("Las columnas restantes (passthrough) se añaden al final")

show_column_mapping(preprocessor)

print(X_train_processed[:1])

## Parte 2. Construyendo el modelo

### Crear la red neuronal

La red neuronal fully connected en pytorch se implementa de forma muy sencilla. Se escpefician los tamaños de las matrices de peso y la función de activación.

In [ ]:
# TODO: Completa la definición del modelo
model = nn.Sequential(
    # TODO: Responde, por que usamos X_train_processed.shape[1]?
    nn.Linear(X_train_processed.shape[1], 6),
    nn.ReLU(),
    ...
    # TODO: Selecciona una función de activación adecuada para salida binaria
)


## Part 3. Entrenamiento

Las redes neuronales se entrenan con descenso de gradiente. En la siguiente sección implementamos descenso de gradiente para aplicar a la red anterior en el conjunto de datos. Para esto se tiene que definir 3 elementos principales:
1. El DataLoader. Este especifica como se van a ir cargando los datos y automatiza el proceso de separlo en minibatches
2. El optimizador. este especifica la regla de actualización de los pesos en su implementación (Por eso aquí se especifica la tasa de aprendizaje)
3. La función de costo. Esta especifica el costo del cual se calculará el gradiente. El optimizador utiliza esta función para guiar la actualización de los pesos. Tenemos que escoger una función apropiada al problema que queremos resolver, en este caso como es clasificación binaria se utiliza el BCELoss.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

print(X_train_processed.shape, y_train.shape)

train_data = []
for i in range(X_train_processed.shape[0]):
   x_i = X_train_processed[i].astype(np.float32) # 11,

   y_i = y_train[i].astype(np.float32)  # scalar
   y_i = np.expand_dims(y_i, axis=-1)  # 1,

   train_data.append([x_i, y_i])
train_loader = DataLoader(train_data, batch_size=7, shuffle=True)

### Descenso de gradeinte
Una vez definido lo anterior simplemente realizamos descenso de gradiente sobre la red

In [ ]:
epochs = 20

for epoch in range(epochs):
	losses = []
	# TODO: Completa el ciclo de entrenamiento
	for x_batch, y_batch in train_loader:
		optimizer.zero_grad()

		# Forward pass
		predictions = ...

		# Calculo del loss
		loss = ...

		# Calculo del gradiente
		...

		# Actualizacion de pesos w = w - lr * grad
		...

		# Agregar loss de este batch a la lista para promediar
		losses.append(loss.item())

		# Agregar un step de valoración si se desea
		# val_step(model, X_val, y_val)
	print(f"epoch {epoch}: training loss {np.mean(losses)}")

## Parte 4. Inferencia

Ahora que vimos como entrenar el modelo intenta predecir para un solo cliente de "prueba". Utiliza la red neuronal para decidir si el siguiente cliente dejará el banco.
- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$ 60000
- Number of Products: 2
- Does this customer have a credit card? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: \$ 50000


**Solución**

In [ ]:
print(feature_cols)
show_column_mapping(preprocessor)

In [ ]:
# Creamos un dato de prueba
# Las primeras son del genero (one hot), despues el pais, las demas en el orden original.
# La cantidad de columnas de genero y pais dependera de como fueron codificadas.
# TODO: inicializa el tensor con datos de prueba
inp_tensor = torch.from_numpy(sc.transform([[]])).float()

# Hacemos la predicción
pred = model(inp_tensor)

# impromimir si es mayor a 0.5 (clase positiva)
print(model(inp_tensor) > 0.5)

Por lo tanto, ¡nuestro modelo de redes neuronales predice que este cliente se queda en el banco!

**Nota importante 1**: Observa que los valores de las características se ingresaron todos dentro de un doble par de corchetes. Esto se debe a que el método predict siempre espera un array 2D como formato de entrada. Y al poner nuestros valores dentro de un doble par de corchetes, hacemos que la entrada sea exactamente un array 2D.

**Nota importante 2**: Observa también que el país "France" no se ingresó como una cadena en la última columna. Esto se debe a que, por supuesto, el método predict espera los valores segun se haya codificado el país, identifica como se codificó Francia. Y ten cuidado de incluir estos valores en las primeras tres columnas, porque las variables dummy siempre se crean en las primeras columnas.

### Prediciendo resultados de validación
Lo comun durante entrenamiento es en cada epoch evaluar TODO el conjunto de validación. En la celda de abajo se muestra una evaluación.

Normalmente esto estaría implementado dentro de un método y estaría dentro del for loop de descenso de gradiente

In [ ]:
X_test_torch = torch.from_numpy(X_test_processed).float()
y_test_torch = torch.from_numpy(y_test).float()

y_pred = model(X_test_torch)
y_pred = (y_pred > 0.5).numpy()

print(y_pred.shape, y_test.shape)

# Resultados, accoracy
y_test_reshaped = y_test.reshape(-1, 1)
print(y_test_reshaped.shape)

# TODO: Calcula la accuracy comparando y_pred y y_test_reshaped
accuracy = ...

print(f"Accuracy de validación: {accuracy * 100:.2f}%")

Otras métricas que podemos calcular son la matriz de confusión o usar las funciones de sklearn para las métricas en lugar de implementarlas directamente

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)